# Collecting responses via Nettskjema API
The following notebook details how to use the Nettskjema API to retrieve responses collected via these webforms. This example combines informatiom from the exiting documentation on this API's structure (https://utv.uio.no/docs/nettskjema/api/), code samples of the python library request, documentation of the r library nettskjemar, and some trial and error.

In order to access the Nettskjema API, you need:

    1) a token (string key) with editing rights to the form in questions
    2) a connection from a suitable IP address (such as on the UIO network)
    3) suitable commands for retrieving the data

## Getting access to the API
In order to access a form through the API, you need to generate an api account for your uio account, generate a unique token with suitable roles for that api account, and grant that api account editing privileges for the form(s) you want to access via the API. This is the link to set up and edit your api account within the nettskjema webinterface: https://nettskjema.no/user/api/index.html

So first set up an account with a simple name and description, then click on that generated account and generate a token with suitable rolls and IP restrictions. Tokens are strings that act as keys so the system knows who is logging in to the API and that they have permission to do specific things. If you are only downloading responses via API (instead of setting up and editing forms), your token needs only the rolls []READ_SUBMISSIONS and []READ_FORMS. If you leave the default IP address range, you can access the API from computers on the UiO network. This include machines logged into remotely (like through VMware Horizon).

When you generate/save the token, the next screen shows you the token string. **COPY AND PASTE THIS INTO A FILE RIGHT AWAY as you will never be able to retrieve it again.** (Though it is easy to just generate another token if needed.) This notebook reads a local directory file called 'nettskjema_token.txt' to get the token string.  

Once you have the token with suitable rolls, click over to your form. Under the rights tab (Rettigheter) should be a list of nettskjema users who have editing rights on this form. So long as you are in one of those accounts, you can add your api account as: "*yourapiaccountname*@api". Note: Once you save, it will show up as "*yourapiaccountname* @ api" but there are no spaces in the user name when you are pasting it in. You can check if the rights have been granted properly by going back to your api account details (under https://nettskjema.no/user/api/index.html) and making sure the form is listed in the Skjemaer table.

Once the rights have been granted, we can use the token to access the API programatically. The API documentation gives examples of curl commands, that can be run from the command like or terminal (https://utv.uio.no/docs/nettskjema/api/). Below are examples of performing the same tasks with python's request library. Also available is an r library developped by UiO research group LCBC to pull response data directly into r data formats, though it does not support extraction of attachments as we need for MusicLab. (more info on *nettskjemar*: https://lcbc-uio.github.io/nettskjemar/index.html) 

# Accessing the API via Request

In [8]:
import requests
import json
import zipfile
# import socket
import os
import base64
#import time

navigate to the directory storing your *nettskjema_token.txt* file

In [9]:
# os.chdir('M:\\finnu\\kant\\div-ritmo-u1')
# os.chdir('M:\\research')
os.listdir()

['PullingNettskjema.ipynb',
 'README.md',
 '.gitignore',
 '.ipynb_checkpoints',
 '.git',
 'nettskjema_token.txt']

In [15]:
f=open('nettskjema_token.txt','r')
TOKEN = f.read()
f.close()

Set up a request session with the token information saved into the authentication settings. This allows us to skip spelling out the token string in this notebook with every API request sent.

In [16]:
session = requests.Session()
session.headers.update({'Authorization': 'Bearer ' + TOKEN})

Test the token by sending a request to see its expiration date.

In [18]:
# confirm the token is working on this IP with the check on expiry date
request_url = "https://nettskjema.no/api/v2/users/admin/tokens/expire-date"
response = session.get(request_url)
response.content

b'{"expireDate":"2022-10-18T17:38:29.000+0200"}'

 If the token is broken or wrong or too old, you will get an error message: 

`b'{"statusCode":400,"message":"Not token authenticated","errors":null,"nestedErrors":null}'`

If the token is recognised, the output will be just the expiry date:

`b'{"expireDate":"2022-10-18T17:38:29.000+0200"}'`

the 'b' before the response string indicates that the API reponse is transmited in bytes that are here being interpreted into ascii. This formate is important to handle carefully when retreiving response attachments that are compressed and encripted. More on that later. 

## Calling for Form metadata

If the token is working, next request information about the form you want to get data from. For this you need the formID number, a unique integer assigned by Nettskjema when the form was created. This is at the end of the form URL (ex: 225781 in https://nettskjema.no/a/225781). The page describing which forms your API account has access to also includes these ID numbers (https://nettskjema.no/user/api/index.html#/user)

The basic request to retreive metadata gives the description of who has access and editing rights, some history of the form, and the content of the form. 

The information returned to requests about forms are json files which can easily be converted into python dictionaries. It is possible to delete and edit forms through the API, but this isn't described here.

The information returned to requests about forms are json files which can easily be converted into python dictionaries.  

In [28]:
# example metadata API request with simple form of one question.
# equivalent curl command
#  $ curl 'https://nettskjema.no/api/v2/forms/225781' -i -X GET -H 'Authorization: Bearer TOKEN'

formID = 225781
request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID)
response = session.get(request_url) # using the request session call which includes the saved API token
form_metadata = json.loads(response.content.decode()) # inteprete recieved string into a python native datatype
form_metadata # show the information output

{'formId': 225781,
 'languageCode': 'en',
 'title': '0 Physically Present Participant Number',
 'deliveryDestination': 'DATABASE',
 'formType': 'DEFAULT',
 'theme': 'DEFAULT',
 'createdBy': {'personId': 616293,
  'username': 'danasw@uio.no',
  'fullName': 'Dana Swarbrick',
  'name': 'Dana Swarbrick',
  'type': 'LOCAL'},
 'modifiedBy': {'personId': 1676908,
  'username': 'finnu@uio.no',
  'fullName': 'Finn Upham',
  'name': 'Finn Upham',
  'type': 'LOCAL'},
 'createdDate': '2021-10-21T09:39:49.000+0200',
 'modifiedDate': '2021-10-28T16:29:27.000+0200',
 'openFrom': '2021-10-27T09:44:45.000+0200',
 'respondentGroup': 'ALL',
 'editorsContactEmail': 'danasw@uio.no',
 'editorsSubmissionEmailType': 'NONE',
 'editors': [{'personId': 1914058,
   'username': 'finnu1@api',
   'fullName': 'finn ritmo access',
   'name': 'finn ritmo access',
   'type': 'API'},
  {'personId': 1676908,
   'username': 'finnu@uio.no',
   'fullName': 'Finn Upham',
   'name': 'Finn Upham',
   'type': 'LOCAL'},
  {'perso

If you do not have access rights to a form, or if you are trying to access the API from an IP address that isn't in the range specified by your token, you get API errors instead, like:

 `{'statusCode': 403,
 'message': 'No access to form with id 225782.',
 'errors': None,
 'nestedErrors': None}`
 
 `{'statusCode': 404,
 'message': 'Could not find form with id 22578.',
 'errors': None,
 'nestedErrors': None}`
 
 

In [49]:
formID = 225782
request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID)
response = session.get(request_url) # using the request session call which includes the saved API token
form_metadata = json.loads(response.content.decode()) # inteprete recieved string into a python native datatype
form_metadata # show the information output

{'statusCode': 403,
 'message': 'No access to form with id 225782.',
 'errors': None,
 'nestedErrors': None}

A slight change in the requestion URL calls instead for metadata on the responses, called "submissions" by the API. The returned info is made into a list of dictionaries with standard information about response (installation ID, submission ID, created and modified dates) as well as all the form responses. 

Responses are listed in reverse chronological order, meaning the last response is the first submission in the list. 

In [34]:
formID = 225781
request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID) + '/submissions' 
response = session.get(request_url) # using the request session call which includes the saved API token
sub_metadata = json.loads(response.content.decode()) # inteprete recieved string into a python native datatype
sub_metadata[:2] # last two responses recieved

[{'submissionId': 16674015,
  'createdDate': '2021-10-27T00:00:00.000+0200',
  'modifiedDate': '2021-10-27T00:00:00.000+0200',
  'delivered': True,
  'answerTime': 0,
  'answers': [{'answerId': 102748640,
    'questionId': 3741786,
    'textAnswer': '40bf6287-e5a4-8a30-8863-cc71c9ef7225'},
   {'answerId': 102748641, 'questionId': 3741787, 'textAnswer': 'Finn'}]},
 {'submissionId': 16653694,
  'createdDate': '2021-10-26T00:00:00.000+0200',
  'modifiedDate': '2021-10-26T00:00:00.000+0200',
  'delivered': True,
  'answerTime': 0,
  'answers': [{'answerId': 102664391,
    'questionId': 3741786,
    'textAnswer': '08ff9d8a-6bc2-897f-24d3-91eb10717741'}]}]

It is possible to request subsets of responses, by the time of submission as well as by submission ID range. Submission IDs increase monotonically, assigned uniquely across all of Nettskjema.no, so a conveninent trick when downloading responses from an active survey is to call for only those submissions recieved since the last time the data was downloaded. 

It is also possible to specify that you only want the submission ID, as sometimes there is no need to download the full responses. 

In [45]:
# how to call submissions from after a certain date

# curl command template"
# $ curl 'https://nettskjema.no/api/v2/forms/8432376/submissions?fields=submissionId&fromDate=2021-01-11T13%3A43%3A17.486%2B0100' -i -X GET -H 'Authorization: Bearer TOKEN'

formID = 141510
# 2021-10-25T08:27:22+01:00 # remembrer URL encoding? : is %3A , + is %2B, https://www.w3schools.com/tags/ref_urlencode.ASP
date = '2021-10-25T08%3A27%3A22.000%2B0100'

request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID) + '/submissions?fields=submissionId&fromDate=' + date

response = session.get(request_url)
subIDs = json.loads(response.content.decode())

print('Submissions since date ' + date + ': ' + str(len(subIDs)))
if len(subIDs)<5:
    print(subIDs)
else:
    print(subIDs[:2])


Submissions since date 2021-10-25T08%3A27%3A22.000%2B0100: 21539
[{'submissionId': 16767882}, {'submissionId': 16767875}]


Forms that do not collect personal information on Nettskjema do not retain dates in a format that can be retrieved this way. You will get this error when trying to restrict the response range by date: 

`{'statusCode': 409, 'message': 'Since the form does not collect personal data, the submissions will not have dates to compare with the fromDate parameter', 'errors': None, 'nestedErrors': None}`

However, because submissionIDs are assigned consecutively across the platform, we can still restrict the call to responses collected since a certain submissionID number. If you are monitoring an active survey, it is convenient to use the ID of the first `[0]` subID from your last API call. 

In [48]:
# how to call submissions from after a certain ID, restricting metadata to submission ID
# curl command template"
# $ curl 'https://nettskjema.no/api/v2/forms/225781/submissions?fields=submissionId&fromSubmissionId=16653694' -i -X GET \
#   -H 'Authorization: Bearer TOKEN'

formID = 225781
submissionID = 16653000
request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID) + '/submissions?fields=submissionId&fromSubmissionId=' + str(submissionID)
response = session.get(request_url)
subIDs = json.loads(response.content.decode())
print('Submissions since subID ' + str(submissionID) + ': ' + str(len(subIDs)))
if len(subIDs)<5: # print error message or top responses 
    print(subIDs)
else:
    print(subIDs[0])


Submissions since subID 16653000: 3
[{'submissionId': 16674015}, {'submissionId': 16653694}, {'submissionId': 16653300}]


# Gathering Musiclab phone sensor data
The above commands cover access to forms that collection information strickly through the webform interface. Apps like Musiclab also gather information in different shapes that are stored by nettskjema as attachments to submissions (responses). These are a bit trickier to retrieve, but still accessible through the API. 

Note: the following cells will not run without permissions for the MusicLab form on Nettskjema, but the shape should be the same for any forms that collects attachments with submissions.

In [53]:
# get metadata on submissions for the music lab app

formID = 141510
submissionID = 16561653
request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID) + '/submissions?fromSubmissionId=' + str(submissionID)
response = session.get(request_url)
subIDs = json.loads(response.content.decode())
print('Submissions since subID ' + str(submissionID) + ': ' + str(len(subIDs)))
if len(subIDs)<5: # print error message or top responses 
    print(subIDs)
else:
    print(subIDs[0])


Submissions since subID 16561653: 23503
{'submissionId': 16767882, 'createdDate': '2021-10-29T08:40:57.000+0200', 'modifiedDate': '2021-10-29T08:40:57.000+0200', 'delivered': True, 'answerTime': 0, 'answers': [{'answerId': 103133022, 'questionId': 1996788, 'textAnswer': 'data.zip', 'attachments': [{'answerAttachmentId': 302365, 'fileName': 'data.zip', 'mediaType': 'application/zip'}]}, {'answerId': 103133021, 'questionId': 1996787, 'textAnswer': 'cfcd73d7-4af9-08e2-aa6a-e73e87bbfbf3'}]}


In order to retrieve the sensor data stored in the submission attachment, we have to call for each file individually and save it appropriately. Here are the essential details from one submission out of the metadata called above. 

In [65]:
subn = 0
print('submissionID : ' + str(subIDs[subn]['submissionId']))
print('Submitting instillation: ' + subIDs[subn]['answers'][1]['textAnswer'])
subIDs[subn]['answers'][0]['attachments'][0]

submissionID : 16767882
Submitting instillation: cfcd73d7-4af9-08e2-aa6a-e73e87bbfbf3


{'answerAttachmentId': 302365,
 'fileName': 'data.zip',
 'mediaType': 'application/zip'}

so to call the attachment for that submission we use the request:

In [84]:
subn = 0 # just calling one as an example

subID = str(subIDs[subn]['submissionId'])
attID = str(subIDs[subn]['answers'][0]['attachments'][0]['answerAttachmentId'])
request_url = 'https://nettskjema.no/api/v2/submissions/' + subID + '/attachments/' + attID
response = session.get(request_url)

att_dets = json.loads(response.content.decode())
print(att_dets)


{'fileName': 'data.zip', 'fileSize': 10637, 'mediaType': 'application/zip', 'content': 'UEsDBAoAAAAAABw1XVOfPYrJwSgAAMEoAAA1AAAAY2ZjZDczZDctNGFmOS0wOGUyLWFhNmEtZTczZTg3YmJmYmYzLmRldmljZU1vdGlvbi5jc3Z0aW1lc3RhbXAsdGltZSx4LHkseixhbHBoYSxiZXRhLGdhbW1hDQoxNjM1NDg5NjUyNTAyLDk2OTcxLDAuODkzOCw3LjE0MSw2Ljg4MDksLTUuMTAxOSwyLjcwNTIsMC4zNTU0DQoxNjM1NDg5NjUyNTE1LDk2OTgzLDAuMTg3Niw3LjY4MDksNi40MzU5LDIuNzA3OCwtMi40MjE2LDAuMTI3OA0KMTYzNTQ4OTY1MjUzMiw5NzAwMCwwLjE1MjgsNy43MDExLDYuMzMxNiw0LjI4NTUsLTAuNjIwNSwwLjc4NTcNCjE2MzU0ODk2NTI1NDgsOTcwMTcsMC4xMTIxLDcuNjcyMiw2LjI5NDQsNS40MzY5LDEuNzEyMywxLjQ5NDMNCjE2MzU0ODk2NTI1NjUsOTcwMzMsMC4wOTU5LDcuNjYzNSw2LjI4MTYsNi4yMjkyLDIuOTI0NiwxLjcNCjE2MzU0ODk2NTI1ODIsOTcwNTAsMC4xODc1LDcuNjU4NSw2LjE5OTksNS44ODA1LDIuNzY0NiwxLjUwMDgNCjE2MzU0ODk2NTI1OTksOTcwNjcsMC4yNTY5LDcuNjc3OCw2LjExNTgsNC45NjUxLDIuMzU4NiwwLjk3NjcNCjE2MzU0ODk2NTI2MTUsOTcwODMsMC4yNzMxLDcuNjkyOSw2LjA5MTQsNC4zNDY0LDEuODA5OCwwLjI0MTkNCjE2MzU0ODk2NTI2MzIsOTcxMDAuMDAwMDAwMDAwMDEsMC4yMjg2LDcuNzI0Myw2LjAyMTYsMy4yMzE0

The nettskjema API returns attachments as encoded zipfiles in byte strings. To make these readable, we need to decode then save the string as a zip file and then unzip them. Thankfully there are python libraries for this.  

First be sure you are in a suitable local folder, then unpack the attachement.

In [67]:
os.mkdir('./Test_API')
os.chdir('Test_API')

In [69]:
subn = 0 # just calling one as an example

subID = str(subIDs[subn]['submissionId'])
attID = str(subIDs[subn]['answers'][0]['attachments'][0]['answerAttachmentId'])
request_url = 'https://nettskjema.no/api/v2/submissions/' + subID + '/attachments/' + attID
response = session.get(request_url)

att_dets = json.loads(response.content.decode())

# write the decoded attachment into a zip file
f=open('data.zip', 'wb')
f.write(base64.b64decode(att_dets['content']))
f.close()

# and then unzip that file, leaving a uniquely titled csv, I hope
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    if not os.path.exists(str(subID)):
        os.mkdir(str(subID))
        zip_ref.extractall('./'+str(subID)) # Not unique filenames so use the unique submission IDs 

print(os.listdir())
os.chdir('./'+str(subID))
print(os.listdir())
os.chdir('..')

['data.zip', '16767882']
['cfcd73d7-4af9-08e2-aa6a-e73e87bbfbf3.deviceMotion.csv']


Here we have a minute recording from a device with the unique installation ID 'cfcd73d7-4a...' in a format that is easy to read. 

The files within the zip are named for the device and information type, but do not include the submission number. This means if they are opened into a folder that already contains a previous recording from the same device, the previously saved recording will be overwritten and lost. To avoid this, the files are unzipped within a folder names for that unique submission. 

Now to collect many at once: 

In [82]:
# pull in the data files and unpack them

newSubs = 0 # count the submission sampled
tic = time.time()
for submis in subIDs[:10]: # just getting 10 as a test
    # first find out the attachment file ID for this submission
    subID = str(submis['submissionId'])
    # if there is an IDed attachment for this submission, get the file
    for subm in submis['answers']:
        if len(subm)>3:
            attID = str(subm['attachments'][0]['answerAttachmentId'])
            request_url = 'https://nettskjema.no/api/v2/submissions/' + subID + '/attachments/' + attID
            response = session.get(request_url)
            newSubs += 1
            att_dets = json.loads(response.content.decode())
            # write the decoded attachment into a zip file
            f=open('data.zip', 'wb')
            f.write(base64.b64decode(att_dets['content']))
            f.close()
            # and then unzip that file, leaving a uniquely titled csv, I hope
            with zipfile.ZipFile('data.zip', 'r') as zip_ref:
                if not os.path.exists(str(subID)):
                    os.mkdir(str(subID))
                    zip_ref.extractall('./'+str(subID)) # if not unique can use the unique submission IDs 

print('time to collect ' + str(newSubs) + ' attachments: ' + str(time.time() - tic))

time to collect 10 attachments: 0.8018779754638672


The files can then be crawled for with suitable information about which submissions related to which instillations, i.e., what can be sewn together in order. 

## compressing submission files
For some forms of storage and retrieval of unzipped data, this folder-per-submission arrangement is really awkward. The following script moves the files from a long list of folders to a single folder while addng the submission number to filenames to preserve uniqueness. 

In [87]:
os.chdir('..')
os.mkdir('CompressedData')

In [88]:
folders = os.listdir('Test_API')
folders 

['.DS_Store',
 '16759222',
 'data.zip',
 '16767882',
 '16758800',
 '16767010',
 '16765118',
 '16766563',
 '16767008',
 '16765068',
 '16767014',
 '16767875']

In [90]:
for subid in folders:
    if subid.startswith('1'):
        filenames = os.listdir('./Test_API/'+str(subid))
        #print(filenames)
        for fn in filenames:
            if fn.endswith('.csv'):
                sourcefile = './Test_API/'+str(subid) + '/' + fn
                targetfile = './CompressedData/'+str(subid) + '.' + fn
                #print(targetfile)
                os.rename(sourcefile,targetfile)
                
os.listdir('./CompressedData/')

['16759222.eee6b926-0668-bd0e-883b-086fea086c3e.geoLocation.csv',
 '16767010.cc61a83b-5b8a-6939-6294-5466a6a9d573.deviceMotion.csv',
 '16767010.cc61a83b-5b8a-6939-6294-5466a6a9d573.geoLocation.csv',
 '16767008.cc61a83b-5b8a-6939-6294-5466a6a9d573.deviceMotion.csv',
 '16767014.cc61a83b-5b8a-6939-6294-5466a6a9d573.geoLocation.csv',
 '16767014.cc61a83b-5b8a-6939-6294-5466a6a9d573.deviceMotion.csv',
 '16765118.97d16b6e-81fa-3a8b-e467-45b7d431861a.deviceMotion.csv',
 '16767882.cfcd73d7-4af9-08e2-aa6a-e73e87bbfbf3.deviceMotion.csv',
 '16765068.97d16b6e-81fa-3a8b-e467-45b7d431861a.deviceMotion.csv',
 '16759222.eee6b926-0668-bd0e-883b-086fea086c3e.deviceMotion.csv',
 '16767875.cfcd73d7-4af9-08e2-aa6a-e73e87bbfbf3.deviceMotion.csv',
 '16766563.aef954a6-ece4-ed70-b034-6c02904b2fe6.geoLocation.csv',
 '16758800.00afa8df-4bef-a5ae-55b2-44933117603a.deviceMotion.csv',
 '16767008.cc61a83b-5b8a-6939-6294-5466a6a9d573.geoLocation.csv',
 '16766563.aef954a6-ece4-ed70-b034-6c02904b2fe6.deviceMotion.csv']

Now, if we need to use sftp to move the data, we don't need to crawl through thousands of folders to find it. 

# Check out other forms
Try to convert other forms response data into convenient formats for analysis

In [93]:
import pandas as pd

In [91]:
formes = [141510,225781,225357,225336,225387,225707,225692,225713,225713,225702,225709,225693,225714,225695,225711]


In [107]:
formID = formes[2]
request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID)
response = session.get(request_url) # using the request session call which includes the saved API token
form_metadata = json.loads(response.content.decode()) # inteprete recieved string into a python native datatype
form_metadata # show the information output

{'formId': 225357,
 'languageCode': 'en',
 'title': '1 Pre-Virtual Concert (English) MusicLab Copenhagen ',
 'deliveryDestination': 'DATABASE',
 'formType': 'DEFAULT',
 'theme': 'DEFAULT',
 'createdBy': {'personId': 616293,
  'username': 'danasw@uio.no',
  'fullName': 'Dana Swarbrick',
  'name': 'Dana Swarbrick',
  'type': 'LOCAL'},
 'modifiedBy': {'personId': 1676908,
  'username': 'finnu@uio.no',
  'fullName': 'Finn Upham',
  'name': 'Finn Upham',
  'type': 'LOCAL'},
 'createdDate': '2021-10-20T09:31:41.000+0200',
 'modifiedDate': '2021-10-29T09:34:59.000+0200',
 'openFrom': '2021-10-27T09:44:31.000+0200',
 'respondentGroup': 'ALL',
 'editorsContactEmail': 'danasw@uio.no',
 'editorsSubmissionEmailType': 'NONE',
 'editors': [{'personId': 573969,
   'username': 'kaylab@uio.no',
   'fullName': 'Kayla Burnim',
   'name': 'Kayla Burnim',
   'type': 'LOCAL'},
  {'personId': 1914058,
   'username': 'finnu1@api',
   'fullName': 'finn ritmo access',
   'name': 'finn ritmo access',
   'type': 

In [117]:
elem = form_metadata['elements']

for m in elem:
    print(str(m['elementId']) + ' ' + m['elementType'])
    if 'questions' in m: 
        for q in m['questions']:
            print('\t Qid: ' + str(q['questionId']))
            print('\t Q: ' + str(q['text']))
            if 'answerOptions' in q:
                for a in q['answerOptions']:
                    print('\t \t' + a['text'])

3484825 QUESTION
	 Qid: 3740598
	 Q: userID
3478911 TEXT
3479045 QUESTION
	 Qid: 3733733
	 Q: Please specify the city you are streaming from:
3479046 QUESTION
	 Qid: 3733734
	 Q: Are you viewing the concert with anyone you know?
3479047 QUESTION
	 Qid: 3733735
	 Q: What is your age?
3479048 RADIO
	 Qid: 3733736
	 Q: What is your gender?
	 	Man
	 	Woman
	 	Other
	 	Choose not to identify
3479049 RADIO
	 Qid: 3733737
	 Q: Which title best describes you?
	 	Tone-deaf
	 	Nonmusician
	 	Music-loving nonmusician
	 	Amateur musician
	 	Serious amateur musician
	 	Semiprofessional musician
	 	Professional musician
3479050 RADIO
	 Qid: 3733738
	 Q: What is your personal relationship to the Danish String Quartet?
	 	I am a relative/friend
	 	I don’t have any particular relation to them
3479051 RADIO
	 Qid: 3733739
	 Q: Are you a fan or admirer of the Danish String Quartet&#39;s music?
	 	1 (Neutral Listener)
	 	2
	 	3
	 	4
	 	5
	 	6
	 	7 (Big Fan/Admirer)
3479310 TEXT
3479052 RADIO
	 Qid: 373374

In [102]:
formID = formes[2]
request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID) + '/submissions?fromSubmissionId=1'
response = session.get(request_url)
subIDs = json.loads(response.content.decode())#eval(response.content.decode())
print(len(subIDs))
if len(subIDs)<5:
    print(subIDs)
else:
    print(subIDs[0])
    

77
{'submissionId': 16674755, 'createdDate': '2021-10-27T00:00:00.000+0200', 'modifiedDate': '2021-10-27T00:00:00.000+0200', 'delivered': True, 'answerTime': 0, 'answers': [{'answerId': 102753918, 'questionId': 3733736, 'answerOptions': [{'answerOptionId': 8624060, 'sequence': 1, 'text': 'Man', 'preselected': False, 'correct': False}]}, {'answerId': 102753916, 'questionId': 3734050, 'answerOptions': [{'answerOptionId': 8624615, 'sequence': 1, 'text': '1 (Does not describe me well)', 'preselected': False, 'correct': False}]}, {'answerId': 102753917, 'questionId': 3734051, 'answerOptions': [{'answerOptionId': 8624624, 'sequence': 5, 'text': '5 (Describes me very well)', 'preselected': False, 'correct': False}]}, {'answerId': 102753910, 'questionId': 3733739, 'answerOptions': [{'answerOptionId': 8624079, 'sequence': 7, 'text': '7 (Big Fan/Admirer)', 'preselected': False, 'correct': False}]}, {'answerId': 102753920, 'questionId': 3733740, 'answerOptions': [{'answerOptionId': 8624599, 'sequ

In [104]:
df = pd.read_json(response.content.decode())
# first flush, answers hid a wealth of information. Would be good to get that broken out 
df.loc[0,'answers']

0     [{'answerId': 102753918, 'questionId': 3733736...
1     [{'answerId': 102753897, 'questionId': 3740598...
2     [{'answerId': 102698549, 'questionId': 3733735...
3     [{'answerId': 102681535, 'questionId': 3733739...
4     [{'answerId': 102678579, 'questionId': 3734049...
                            ...                        
72    [{'answerId': 102498788, 'questionId': 3733735...
73    [{'answerId': 102203932, 'questionId': 3734052...
74    [{'answerId': 102203910, 'questionId': 3733737...
75    [{'answerId': 102052489, 'questionId': 3740598...
76    [{'answerId': 102047548, 'questionId': 3733735...
Name: answers, Length: 77, dtype: object

In [105]:
df.loc[0,'answers']

[{'answerId': 102753918,
  'questionId': 3733736,
  'answerOptions': [{'answerOptionId': 8624060,
    'sequence': 1,
    'text': 'Man',
    'preselected': False,
    'correct': False}]},
 {'answerId': 102753916,
  'questionId': 3734050,
  'answerOptions': [{'answerOptionId': 8624615,
    'sequence': 1,
    'text': '1 (Does not describe me well)',
    'preselected': False,
    'correct': False}]},
 {'answerId': 102753917,
  'questionId': 3734051,
  'answerOptions': [{'answerOptionId': 8624624,
    'sequence': 5,
    'text': '5 (Describes me very well)',
    'preselected': False,
    'correct': False}]},
 {'answerId': 102753910,
  'questionId': 3733739,
  'answerOptions': [{'answerOptionId': 8624079,
    'sequence': 7,
    'text': '7 (Big Fan/Admirer)',
    'preselected': False,
    'correct': False}]},
 {'answerId': 102753920,
  'questionId': 3733740,
  'answerOptions': [{'answerOptionId': 8624599,
    'sequence': 5,
    'text': '5 (Describes me very well)',
    'preselected': False,
  

In [106]:
df.loc[1,'answers']

[{'answerId': 102753897,
  'questionId': 3740598,
  'textAnswer': '83530137-77f7-afa8-b403-eee29ba8198f'},
 {'answerId': 102753904,
  'questionId': 3733739,
  'answerOptions': [{'answerOptionId': 8624079,
    'sequence': 7,
    'text': '7 (Big Fan/Admirer)',
    'preselected': False,
    'correct': False}]},
 {'answerId': 102753895,
  'questionId': 3733737,
  'answerOptions': [{'answerOptionId': 8624066,
    'sequence': 3,
    'text': 'Music-loving nonmusician',
    'preselected': False,
    'correct': False}]},
 {'answerId': 102753892, 'questionId': 3733734, 'textAnswer': 'No'},
 {'answerId': 102753903,
  'questionId': 3733738,
  'answerOptions': [{'answerOptionId': 8624072,
    'sequence': 2,
    'text': 'I don’t have any particular relation to them',
    'preselected': False,
    'correct': False}]},
 {'answerId': 102753902,
  'questionId': 3733736,
  'answerOptions': [{'answerOptionId': 8624060,
    'sequence': 1,
    'text': 'Man',
    'preselected': False,
    'correct': False}]}

# old notes
Username
Short description
Personal owners
Last active

finnuritmo @ api

finnuritmo @ api

New token created
Token: dar7qsem18ljlqc0ntg8ivbble69nf0g26n8177pk1hlskstanj3gfu78se8q12pro889535p9quqi78akr4hhajijnkj6l0b79aflgdgehlb0ht50870ggkm9t5rnk6fr86
Save the token as it is not stored in the Web form.

The token can be used for the following API calls:
WRITE_FORMS
WRITE_INVITATIONS
READ_SUBMISSIONS
READ_INVITATIONS
READ_FORMS
Allowed IP addresses : 129.240.0.0/16, 193.157.0.0/16

193.157.119.236

https://nettskjema.no/user/api/index.html

https://lcbc-uio.github.io/nettskjemar/articles/auth_setup.html

https://realpython.com/api-integration-in-python/#rest-and-python-consuming-apis

https://docs.python-requests.org/en/latest/user/authentication/



https://nettskjema.no/a/141510


$ curl 'https://nettskjema.no/api/v2/submissions/5136791' -i -X GET \
    -H 'Authorization: Bearer TOKEN'


curl 'https://nettskjema.no/api/v2/users/admin/tokens/expire-date' -i -X GET \ -H 'Authorization: Bearer TOKEN'



$ curl 'https://nettskjema.no/api/v2/submissions/141510' -i -X GET -H 'Authorization: finnuritmo @ api TOKEN'


curl 'https://nettskjema.no/api/v2/users/admin/tokens/expire-date' -i -X GET \ -H 'Authorization: Bearer TOKEN'





*******
base64 --decode customname1.txt > datazip.zip

base64 0c6a885b-de29-a128-1480-fe98e4a06f69.geoLocation.csv  > output.txt

/Desktop/Current_Projects/Resp_Mobile_recording/DataJockying/data(8).zip



